Imported the libraries

In [1]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Download the file

In [3]:
torontodf = pd.read_csv('TorontoPostalCode2.csv', delimiter = ';')
torontodf.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Removal of empty cells in the Borough column. Change the name 'Not assigned' in 'NaN', in the 'Borough' column.

In [4]:
toronto = torontodf.replace({'Borough': 'Not assigned'},np.NaN)

In [5]:
torontodfSansNA = toronto.dropna(subset=['Borough'],axis=0)

Check if the Postal Code is unique or not

In [6]:
torontodoubled = torontodfSansNA['Postal code'].unique().shape
if (torontodfSansNA.shape[0]==torontodoubled[0]):
     print('Postal Code is OK, none of its values is doubled')
else:
     print ('Some incongruences found, please check consistency')

Postal Code is OK, none of its values is doubled


Check if the Neighborhood colums has 'Not assigned' values

In [7]:
torontodfSansNA.isna().sum()

Postal code     0
Borough         0
Neighborhood    0
dtype: int64

The number of rows of the dataframe :

In [8]:
torontodfSansNA.shape

(103, 3)

Download the coordinates file.

In [9]:
torontoCoor = pd.read_csv('Geospatial_Coordinates.csv', delimiter = ',')
torontoCoor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Combined the two files (Postal Code + Coordinates)

In [10]:
torontoTotal = torontodfSansNA.merge(torontoCoor, left_on="Postal code", right_on ="Postal Code",how='outer')        

In [11]:
torontoCoord=torontoTotal.drop("Postal Code", axis=1)

In [16]:
torontoCoord.head(12)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


Coordinates of Toronto :

In [36]:
address = 'Toronto, T'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Create map of Toronto with Borough markers only :

In [37]:
mapToronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough in zip(torontoCoord['Latitude'], torontoCoord['Longitude'], torontoCoord['Borough']):
    label = ' {}'.format( borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(mapToronto)  
    
mapToronto